#### Named Entity Recognition (NER)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nastyachizhikova/doc_test/blob/main/source/notebooks/NER.ipynb)

# Table of contents 

1. [Introduction to the task](#1.-Introduction-to-the-task)

2. [Get started with the model](#2.-Get-started-with-the-model)

3. [Use the model for prediction](#3.-Use-the-model-for-prediction)

    3.1. [Predict using Python](#3.1-Predict-using-Python)
    
    3.2. [Predict using CLI](#3.2-Predict-using-CLI)
     
4. [Train the model on your data](#4.-Train-the-model-on-your-data)
    
    4.1. [Train your model from Python](#4.1-Train-your-model-from-Python)
    
    4.2. [Train your model from CLI](#4.2-Train-your-model-from-CLI)
    
5. [Evaluate](#5.-Evaluate)
    
    5.1. [Evaluate from Python](#5.1-Evaluate-from-Python)
    
    5.2. [Evaluate from CLI](#5.2-Evaluate-from-CLI)
    
6. [Models list](#7.-Models-list)

7. [NER-tags list](#7.-NER-tags-list)


# 1. Introduction to the task

**Named Entity Recognition (NER)** is a task of assigning a tag (from a predefined set of tags) to each token in a given sequence. In other words, NER-task consists of identifying named entities in the text and classifying them into types (e.g. person name, organization, location etc). 

**BIO encoding schema** is usually used in NER task. It uses 3 tags: B for the beginning of the entity, I for the inside of the entity, and O for non-entity tokens. The second part of the tag stands for the entity type.

Here is an example of a tagged sequence:

| Elon | Musk | founded | Tesla| in | 2003 | . |
| --- | --- | --- | --- | --- | --- | --- |
| B-PER | I-PER | O | B-ORG | O | B-DATE | O |

Here we can see three extracted named entities: *Elon Musk* (which is a person's name), *Tesla* (which is a name of an organization) and *2003* (which is a date). To see more examples try out our [Demo](https://demo.deeppavlov.ai/#/en/ner).

The list of possible types of NER entities may vary depending on your dataset domain. The list of tags used in DeepPavlov's models can be found in the [table](#5.-NER-tags-list).

# 2. Get started with the model

First make sure you have the DeepPavlov Library installed.
[More info about the first installation](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Get%20Started%20with%20DeepPavlov.html)

In [1]:
!pip install --q deeppavlov

Then make sure that all the required packages for the model are installed.

In [ ]:
!python -m deeppavlov install ner_ontonotes_bert_torch

`ner_ontonotes_bert_torch` here is the name of the model's *config_file*. [What is a Config File?](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Config%20File.html) 

Configuration file defines the model and describes its hyperparameters. To use another model, change the name of the *config_file* here and further.
The full list of NER models with their config names can be found in the [table](#6.-Models-list).


# 3. Use the model for prediction

## 3.1 Predict using Python

After [installing](#2.-Get-started-with-the-model) the model, build it from the config and predict.

In [ ]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_ontonotes_bert_torch, download=True)

**Input**: List[sentences]

**Output**: List[tokenized sentences, corresponding NER-tags]

In [ ]:
ner_model(['Bob Ross lived in Florida', 'Elon Musk founded Tesla'])

[[['Bob', 'Ross', 'lived', 'in', 'Florida'],
  ['Elon', 'Musk', 'founded', 'Tesla']],
 [['B-PERSON', 'I-PERSON', 'O', 'O', 'B-GPE'],
  ['B-PERSON', 'I-PERSON', 'O', 'B-ORG']]]

## 3.2 Predict using CLI

You can also get predictions in an interactive mode through CLI.

In [ ]:
! python -m deeppavlov interact ner_ontonotes_bert_torch -d

`-d` is an optional download key (alternative to `download=True` in Python code). The key `-d` is used to download the pre-trained model along with embeddings and all other files needed to run the model. 

Or make predictions for samples from *stdin*.

In [ ]:
! python -m deeppavlov predict ner_ontonotes_bert_torch -f <file-name>

# 4. Evaluate 

There are two metrics that are used to evaluate a NER model in DeepPavlov:

`ner_f1` is measured on the entity-level (actual text spans should match exactly)

`ner_token_f1` is measured on a token level (correct tokens from not fully extracted entities will still be counted as TPs)

## 4.1 Evaluate from Python

In [ ]:
from deeppavlov import evaluate_model

model = evaluate_model(configs.ner.ner_ontonotes_bert_torch, download=True)

## 4.1 Evaluate from CLI

In [ ]:
! python -m deeppavlov evaluate ner_ontonotes_bert_torch

# 5. Train the model on your data

## 5.1 Train your model from Python

### Provide your data path

To train the model on your data, you need to change the path to the training data in the *config_file*. 

Parse the *config_file* and change the path to your data from Python.

In [ ]:
from deeppavlov import configs, train_model
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config(configs.ner.ner_ontonotes_bert_torch)

#  dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/ontonotes/


Provide a *data_path* to your own dataset. 

In [7]:
# download and unzip a new example dataset
!wget http://files.deeppavlov.ai/deeppavlov_data/conll2003_v2.tar.gz
!tar -xzvf "conll2003_v2.tar.gz"

In [6]:
# provide a path to the train file
model_config["dataset_reader"]["data_path"] = "contents/train.txt"


### Train dataset format

To train the model, you need to have a txt-file with a dataset in the following format:

```
EU B-ORG
rejects O
the O
call O
of O
Germany B-LOC
to O
boycott O
lamb O
from O
Great B-LOC
Britain I-LOC
. O

China B-LOC
says O
time O
right O
for O
Taiwan B-LOC
talks O
. O
```


The source text is **tokenized** and **tagged**. For each token, there is a tag with BIO markup. Tags are separated from tokens with **whitespaces**. Sentences are separated with **empty lines**.


### Train the model using new config

In [ ]:
ner_model = train_model(model_config)

Use your model for prediction.

In [ ]:
ner_model(['Bob Ross lived in Florida', 'Elon Musk founded Tesla'])

[[['Bob', 'Ross', 'lived', 'in', 'Florida'],
  ['Elon', 'Musk', 'founded', 'Tesla']],
 [['B-PERSON', 'I-PERSON', 'O', 'O', 'B-GPE'],
  ['B-PERSON', 'I-PERSON', 'O', 'B-ORG']]]

## 5.2 Train your model from CLI

In [ ]:
! python -m deeppavlov train ner_ontonotes_bert_torch

# 6. Models list

The table presents a list of all of the NER-models available in DeepPavlov Library.

| Config name  | Dataset | Language | Model Size | F1 score (ner_f1) | F1 score (ner_f1_token)
| :--- | --- | --- | --- | --- | ---: |
| ner_case_agnostic_mdistilbert| [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003)   | En | 1.6 GB | 89.9 | 91.6 |
| ner_conll2003_bert | [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003) | En | 1.3 GB | **91.9** | **93.4** |
| ner_ontonotes_bert | [OntoNotes](https://paperswithcode.com/dataset/ontonotes-5-0) | En | 1.3 GB | 89.2 | 92.7 |
| ner_collection3_bert | [Collection3](https://www.researchgate.net/publication/313808701_Combining_Knowledge_and_CRF-Based_Approach_to_Named_Entity_Recognition_in_Russian) | Ru | 2.1 GB | **98.5** | **98.9** |
| ner_rus_bert | [Collection3](https://www.researchgate.net/publication/313808701_Combining_Knowledge_and_CRF-Based_Approach_to_Named_Entity_Recognition_in_Russian) | Ru | 2.1 GB | 97.6 | 98.5 |
| ner_rus_convers_distilrubert_2L | [Collection-rus](https://www.researchgate.net/publication/313808701_Combining_Knowledge_and_CRF-Based_Approach_to_Named_Entity_Recognition_in_Russian) | Ru | 1.3 GB | 92.9 | 96.6 |
| ner_rus_convers_distilrubert_6L | [Collection-rus](https://www.researchgate.net/publication/313808701_Combining_Knowledge_and_CRF-Based_Approach_to_Named_Entity_Recognition_in_Russian) | Ru | 1.6 GB | 96.7 | 98.5 |
| ner_rus_bert_probas | [Wiki-NER-rus](https://aclanthology.org/I17-1042/) | Ru | 2.1 GB | 72.6 | 79.5 |
| ner_ontonotes_bert_mult | [OntoNotes](https://paperswithcode.com/dataset/ontonotes-5-0) | Multi | 2.1 GB | 88.9 | 92.0 |


# 7. NER-tags list

The table presents a list of all of the NER entity tags used in DeepPavlov's NER-models.

|              |                                                        |
| ------------ | ------------------------------------------------------ |
| **PERSON**       | People including fictional                             |
| **NORP**         | Nationalities or religious or political groups         |
| **FACILITY**     | Buildings, airports, highways, bridges, etc.           |
| **ORGANIZATION** | Companies, agencies, institutions, etc.                |
| **GPE**          | Countries, cities, states                              |
| **LOCATION**     | Non-GPE locations, mountain ranges, bodies of water    |
| **PRODUCT**      | Vehicles, weapons, foods, etc. (Not services)          |
| **EVENT**        | Named hurricanes, battles, wars, sports events, etc.   |
| **WORK OF ART**  | Titles of books, songs, etc.                           |
| **LAW**          | Named documents made into laws                         |
| **LANGUAGE**     | Any named language                                     |
| **DATE**         | Absolute or relative dates or periods                  |
| **TIME**         | Times smaller than a day                               |
| **PERCENT**      | Percentage (including “%”)                             |
| **MONEY**        | Monetary values, including unit                        |
| **QUANTITY**     | Measurements, as of weight or distance                 |
| **ORDINAL**      | “first”, “second”                                      |
| **CARDINAL**     | Numerals that do not fall under another type           |